In [1]:
import warnings

import matplotlib as mpl
import numpy as np
from IPython import get_ipython

import pymor.tools.random

ip = get_ipython()
if ip is not None:
#    ip.run_line_magic('matplotlib', 'inline')
    ip.run_line_magic('matplotlib', 'ipympl')

warnings.filterwarnings('ignore', category=UserWarning, module='torch')

pymor.tools.random._default_random_state = None

mpl.rcParams['figure.facecolor'] = (1.0, 1.0, 1.0, 0.0)

In [2]:
from pymor.basic import *

p = thermal_block_problem([2,2])
p = p.with_(outputs=[('l2', ConstantFunction(1., 2))])#, ('l2', ExpressionFunction('x[0] >= 0.5', 2))])
fom, _ = discretize_stationary_cg(p)

parameter_space = fom.parameters.space((0.1, 1))

Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))

In [3]:
#interact(fom, parameter_space)

In [4]:
from pymor.models.hierarchy import AdaptiveModelHierarchy
from pymor.reductors.neural_network import NeuralNetworkReductor

In [5]:
rb_reductor = CoerciveRBReductor(
    fom,
    product=fom.h1_0_semi_product,
    coercivity_estimator=ExpressionParameterFunctional('min(diffusion)', fom.parameters)
)

def reduction_rb(training_data, len_previous_training_data, models, reductors):
    U = fom.solution_space.empty(reserve=len(training_data))
    for _, u in training_data[len_previous_training_data:]:
        U.append(u)
    RB, _ = pod(U, product=fom.h1_0_semi_product)
    reductors[0].extend_basis(RB)
    return reductors[0].reduce()

def post_reduction_rb(training_data, models, reductors):
    return []

tolerance = 5e-3

# Settings for the two-stage hierarchy
models = [rb_reductor.reduce(), fom]
model_names = ['RB-ROM', 'FOM']
reductors = [rb_reductor]
reduction_methods = [reduction_rb]
post_reduction_methods = [post_reduction_rb]
training_frequencies = [1]

two_stage_hierarchy = AdaptiveModelHierarchy(models, reductors, reduction_methods, post_reduction_methods,
                                             training_frequencies, tolerance, visualizer=fom.visualizer,
                                             name='Two-stage model hierarchy')

Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))

In [6]:
from pymor.models.interact import interact_model_hierarchy

In [7]:
#interact_model_hierarchy(two_stage_hierarchy, parameter_space, model_names)

In [8]:
ml_reductor = NeuralNetworkReductor(fom=fom, training_set=None, validation_set=None,
                                    ann_mse=None, pod_params={'product': fom.h1_0_semi_product},
                                    scale_inputs=False, scale_outputs=False)

def reduction_ml(training_data, len_previous_training_data, models, reductors):
    rb_rom = models[1]
    rb_reductor = reductors[1]
    ml_reductor = reductors[0]
    error_estimator = rb_rom.error_estimator
    ml_reductor.reduced_basis = rb_reductor.bases['RB']
    red_dim = len(ml_reductor.reduced_basis)
    training_data = [(mu, np.pad(dat.to_numpy()[0], (0, red_dim - len(dat.to_numpy()[0])),
                                 mode='constant', constant_values=0.))
                     for (mu, dat) in training_data]
    ml_reductor.training_data = training_data
    ml_rom = ml_reductor.reduce(restarts=2, log_loss_frequency=10, recompute_training_data=False,
                                recompute_validation_data=True)
    return ml_rom.with_(error_estimator=error_estimator)

def post_reduction_ml(training_data, models, reductors):
    return []

def post_reduction_rb(training_data, models, reductors):
    return [models[0]]
    #return [reduction_ml(training_data[1], None, models, reductors)]

models = [None, rb_reductor.reduce(), fom]
model_names = ['ML-ROM', 'RB-ROM', 'FOM']
reductors = [ml_reductor, rb_reductor]
reduction_methods = [reduction_ml, reduction_rb]
post_reduction_methods = [post_reduction_ml, post_reduction_rb]
training_frequencies = [20, 1]

three_stage_hierarchy = AdaptiveModelHierarchy(models, reductors, reduction_methods, post_reduction_methods,
                                               training_frequencies, tolerance)

In [9]:
interact_model_hierarchy(three_stage_hierarchy, parameter_space, model_names,
                         visualizer=fom.visualize)

Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))